In [1]:
import pandas as pd

#### Padding Summary¶
* Valid Padding (No Padding)<br>
    Output size < Input Size
* Same Padding (Zero Padding)<br>
    Output size = Input Size

##### Dimension calculation
* out_dim  $$
O = \frac{W - K + 2P}{S}  + 1\\
$$

* O:output height/width
* W:input height/length
* K: filter size(kernel size)
* P: padding =$$
P = \frac{K - 1}{2}\\
$$
* S: stride

## load mnist model

In [3]:

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets

train_dataset = datasets.MNIST(root = './data',
                                   train = True,
                                   transform = transforms.ToTensor(),download = True)


test_dataset = datasets.MNIST(root = './data',
                                   train = False,
                                   transform = transforms.ToTensor(),download = True)


9913344it [01:41, 97657.12it/s]                              


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Using downloaded and verified file: ./data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Using downloaded and verified file: ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Using downloaded and verified file: ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...


/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [9]:
print(train_dataset.train_data.shape, train_dataset.train_data.shape)

torch.Size([60000, 28, 28]) torch.Size([60000, 28, 28])


In [61]:

from torch.utils.data import DataLoader
batch_size = 128
n_iters = 3000
num_epochs = int(n_iters/(len(train_dataset)/batch_size) ) + 3


train_loader = DataLoader(train_dataset,
                         shuffle = True, 
                         batch_size = batch_size)


test_loader = DataLoader(train_dataset,
                         shuffle = True, 
                         batch_size = batch_size)

## convolution calculations

#### output calculation for convolution layer
$$ 
O = \frac{W - K + 2P}{S}  + 1\\
$$
* K(kernel_size) = 5(5*5)
* S(stride) = 1
* P = (k - 1)/2 = 2

#### output dimension for pooling layer
$$ 
O = \frac{W - K}{S} + 1\\
$$
* kernel size = 2
* stride = kernel size (by defualt in pytorch) = 2




Here, 
* output of the convolution layer
$$
O = \frac{28 - 5 + 2*2}{1} + 1\\ = 28
$$

* output of the pooling layer
$$
O = \frac{28 - 2}{2} + 1\\ = 14
$$

<img src="./data/cnn10-2n.png" alt="image info" />


## Model creation


## model 1: with padding(same padding)

In [71]:
class CNN_model(nn.Module):
    def __init__(self):
        
        
        
        super(CNN_model, self).__init__()
        
        # layer1: convolutional layer 
        self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 5, stride = 1, padding = 2)
        self.relu1 = nn.ReLU()
        
        # layer2: max pooling
        self.maxpool1 = nn.MaxPool2d(kernel_size  = 2)
        
        # layer3: convolutional layer 
        self.cnn2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 1, padding = 2)
        self.relu2 = nn.ReLU()
        
        # layer4: max pooling
        self.maxpool2 = nn.MaxPool2d(kernel_size  = 2)
        
        # output layer: Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 7 * 7, 10) 
        
        
    def forward(self, x):
        
        out = self.cnn1(x)
        out = self.relu1(out)

        out = self.maxpool1(out)

        
        out = self.cnn2(out)
        out = self.relu2(out)

        
        out = self.maxpool2(out)

        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)

        return out
        
        
    
model = CNN_model()        

In [72]:
#### cheking  model parameters
print("conv layer1 kernel: ", list(model.parameters())[0].shape)
print("maxpooling1: ", list(model.parameters())[1].shape)
print("conv layer2 kernel: ", list(model.parameters())[2].shape)
print("maxpooling2: ", list(model.parameters())[3].shape)
print("output linear: ", list(model.parameters())[4].shape)

conv layer1 kernel:  torch.Size([16, 1, 5, 5])
maxpooling1:  torch.Size([16])
conv layer2 kernel:  torch.Size([32, 16, 5, 5])
maxpooling2:  torch.Size([32])
output linear:  torch.Size([10, 1568])


#### instantiating the loss fucntion and the optimizer for the model

In [77]:
import torch.optim as optim
learning_rate = 0.01
optimizer = optim.SGD(model.parameters(), lr = learning_rate )
criterion = nn.CrossEntropyLoss()

## model 2: no padding(valid padding)

<img src = "./data/cnn10-6n.png">

In [74]:
class CNN_model(nn.Module):
    def __init__(self):
        
        
        
        super(CNN_model, self).__init__()
        
        # layer1: convolutional layer 
        self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 5, stride = 1, padding = 0)
        self.relu1 = nn.ReLU()
        
        # layer2: max pooling
        self.maxpool1 = nn.MaxPool2d(kernel_size  = 2)
        
        # layer3: convolutional layer 
        self.cnn2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 1, padding = 0)
        self.relu2 = nn.ReLU()
        
        # layer4: max pooling
        self.maxpool2 = nn.MaxPool2d(kernel_size  = 2)
        
        # output layer: Fully connected 1 (readout)
        self.fc1 = nn.Linear(32*4*4, 10) 
        
        
    def forward(self, x):
        
        ##### conv  layer 1 ##### 
        out = self.cnn1(x)
        out = self.relu1(out)
        ######################### 
        
        ##### max pool 1 ##### 
        out = self.maxpool1(out)
        ######################### 
        
        ##### conv  layer 2 #####         
        out = self.cnn2(out)
        out = self.relu2(out)
        ######################### 

        ##### max pool 2 ##### 
        out = self.maxpool2(out)
        #########################
        
        # reshaping the output
        out = out.view(out.size(0), -1)

        ##### output layer: linear ##### 
        out = self.fc1(out)
        ################################

        return out
        
        
    
model = CNN_model()        

#### model traning module

In [96]:
iter = 0;
for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_loader):
#         no need to reshape images for convluation network
        
        images = images.requires_grad_()
        # clearing the optimizer gradient
        optimizer.zero_grad()
        
        # forward pass to get the output
        output = model(images)
        
        # calculate loss: softmax => cross entropy loss
        loss = criterion(output, labels)
        
        # getting gradients
        loss.backward()
        
        # updating the weights with 
        optimizer.step()
        
        iter += 1
        
        # validating the current model
        if iter%500 == 0:
            correct = 0
            total = 0
            
            
            for images, labels in test_loader:
                images = images.requires_grad_()
                
                outputs = model.forward(images)
                # it doesnt make much differnce if we use or not use softmax function here
                outputs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)
                
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
                
                
            accuracy = 100*(correct/total)
            

#           Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))


                
                
                
        

Iteration: 500. Loss: 0.09718340635299683. Accuracy: 97.15333557128906
Iteration: 1000. Loss: 0.13340891897678375. Accuracy: 97.33833312988281
Iteration: 1500. Loss: 0.08473212271928787. Accuracy: 97.3933334350586
Iteration: 2000. Loss: 0.17217475175857544. Accuracy: 97.34667205810547
Iteration: 2500. Loss: 0.13855279982089996. Accuracy: 97.45833587646484
Iteration: 3000. Loss: 0.062202829867601395. Accuracy: 97.77833557128906
Iteration: 3500. Loss: 0.04042651876807213. Accuracy: 97.76166534423828
Iteration: 4000. Loss: 0.06984707713127136. Accuracy: 97.84833526611328
